In [1]:
import pandas as pd

df = pd.read_csv('D1BaseballTrackman.csv')

In [12]:
team_abbreviations = [
    "TEX_AGG", "ARK_RAZ", "LSU_TIG", "OLE_REB", "MSU_BDG",
    "ALA_CRI", "AUB_TIG", "VAN_COM", "TEN_VOL", "GEO_BUL",
    "SOU_GAM", "FLA_GAT", "KEN_WIL", "MIZ_TIG"
]

# Filter the dataframe for the specific team abbreviations in both BatterTeam and PitcherTeam
df_filtered = df[df['BatterTeam'].isin(team_abbreviations) & df['PitcherTeam'].isin(team_abbreviations)]

df_filtered.head()

,Date,TaggedPitchType,PlateLocSide,PlateLocHeight,League,GameID,BatterTeam,PitcherTeam,Pitcher,Batter,...,RelHeight,SpinRate_1,Inning,Events,mean_run_value,Count,event_dpdrv,Event_Value,dPDRV,RunValue
190,2022-05-20,Sinker,-0.18199,1.89736,SEC,20220520-SwellThomasStadium-1,ARK_RAZ,ALA_CRI,"McNairy, Jacob","Slavens, Brady",...,5.74620,2017.180021,4,FoulBall,NaN,0-0,FoulBall_0_0,-0.075468,0.157692,-0.075468
191,2022-05-21,Fastball,-0.00980,3.32762,SEC,20220521-SwellThomasStadium-1,ALA_CRI,ARK_RAZ,"Starks, Gabriel","Pinckney, Andrew",...,6.53324,2344.992241,8,FoulBall,NaN,2-2,FoulBall_2_2,-0.000030,0.277458,-0.000030
195,2022-05-27,Undefined,0.64624,2.34563,SEC,20220527-HooverMetropolitan-3,ALA_CRI,TEX_AGG,"Cortez, Christopher","Seidl, Tommy",...,5.44849,2080.063487,5,Out,-0.376228,1-2,Out_1_2,NaN,-0.235294,-0.235294
197,2022-03-18,ChangeUp,-0.22315,1.44980,SEC,20220318-SwellThomasStadium-1,FLA_GAT,ALA_CRI,"McMillan, Garrett","Rivera, Josh",...,6.02012,1510.259379,2,Single,0.576249,1-1,Single_1_1,NaN,-0.107761,-0.107761
198,2023-03-24,Sinker,0.12527,1.47576,SEC,20230324-SwellThomasStadium-1,KEN_WIL,ALA_CRI,"Quick, Riley","Burkes, Devin",...,5.45711,2100.354469,12,StrikeCalled,NaN,1-1,StrikeCalled_1_1,-0.112217,NaN,-0.112217


In [3]:
# Function to determine if the batter gets on base
def is_on_base(row):
    on_base_events = ['Single', 'Double', 'Triple', 'Home Run', 'Walk', 'Error']
    return row['PlayResult'] in on_base_events or (row['PlayResult'] == 'Undefined' and row['Events'] == 'HitByPitch')

In [6]:
# Group by game ID, inning, and top/bottom
half_innings = df_filtered.groupby(['GameID', 'Inning', 'Top_Bottom'])

# Initialize variables
total_runs = 0
half_innings_with_first_batter_on_base = 0
half_innings_with_runs = 0

for name, group in half_innings:
    first_batter = group.iloc[0]
    runs_scored = group['RunsScored'].sum()

    if is_on_base(first_batter):
        half_innings_with_first_batter_on_base += 1
        total_runs += runs_scored

        if runs_scored > 0:
            half_innings_with_runs += 1

# Calculate average runs scored
if half_innings_with_first_batter_on_base > 0:
    avg_runs = total_runs / half_innings_with_first_batter_on_base
    chance_of_scoring = (half_innings_with_runs / half_innings_with_first_batter_on_base) * 100
else:
    avg_runs = 0
    chance_of_scoring = 0

In [5]:
print(df_filtered.columns)

Index(['Date', 'TaggedPitchType', 'PlateLocSide', 'PlateLocHeight', 'League',
       'GameID', 'BatterTeam', 'PitcherTeam', 'Pitcher', 'Batter', 'Balls',
       'Outs', 'Strikes', 'RunsScored', 'Top_Bottom', 'Time', 'ExitSpeed',
       'Angle', 'Direction', 'HitSpinRate', 'PositionAt110X', 'PositionAt110Y',
       'BatterSide', 'PitcherThrows', 'PitcherId', 'PitchCall', 'PlayResult',
       'KorBB', 'RelSpeed', 'InducedVertBreak', 'HorzBreak', 'Tilt',
       'SpinRate', 'VertApprAngle', 'Extension', 'RelSide', 'RelHeight',
       'SpinRate_1', 'Inning', 'Events', 'mean_run_value', 'Count',
       'event_dpdrv', 'Event_Value', 'dPDRV', 'RunValue'],
      dtype='object')


In [7]:
# Round chance_of_scoring to 1 decimal place
avg_runs = round(avg_runs, 2)
chance_of_scoring_rounded = round(chance_of_scoring, 1)

# Print the statement
print(f"In the SEC, when the leadoff hitter of an inning reached, the average runs per inning was {avg_runs} and the chance of scoring a run was {chance_of_scoring_rounded}%.")

In the SEC, when the leadoff hitter of an inning reached, the average runs per inning was 1.12 and the chance of scoring a run was 50.1%.


In [8]:

# Function to determine if the batter does not get on base
def does_not_get_on_base(row):
    on_base_events = ['Single', 'Double', 'Triple', 'Home Run', 'Walk', 'Error']
    return not (row['PlayResult'] in on_base_events or (row['PlayResult'] == 'Undefined' and row['Events'] == 'HitByPitch'))

In [9]:
# Initialize variables for calculations
total_runs = 0
half_innings_with_first_batter_not_on_base = 0
half_innings_with_runs = 0

# Group by game ID, inning, and top/bottom for unique half innings
for name, group in df_filtered.groupby(['GameID', 'Inning', 'Top_Bottom']):
    first_batter = group.iloc[0]
    runs_scored = group['RunsScored'].sum()

    if does_not_get_on_base(first_batter):
        half_innings_with_first_batter_not_on_base += 1
        total_runs += runs_scored

        if runs_scored > 0:
            half_innings_with_runs += 1

# Calculate average runs scored and chance of scoring
if half_innings_with_first_batter_not_on_base > 0:
    avg_runs = total_runs / half_innings_with_first_batter_not_on_base
    chance_of_scoring = (half_innings_with_runs / half_innings_with_first_batter_not_on_base) * 100
else:
    avg_runs = 0
    chance_of_scoring = 0

In [10]:
# Round chance_of_scoring to 1 decimal place
avg_runs = round(avg_runs, 2)
chance_of_scoring_rounded = round(chance_of_scoring, 1)

# Print the statement
print(f"In the SEC, when the leadoff hitter of an inning was retired, the average runs per inning was {avg_runs} and the chance of scoring was {chance_of_scoring_rounded}%.")

In the SEC, when the leadoff hitter of an inning was retired, the average runs per inning was 0.59 and the chance of scoring was 30.1%.
